## Project 1: NYC Resteraunt Violations Cleaning Up
### Jacob Minkin

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
sns.set_style('darkgrid')

%matplotlib inline

## Set-Up

In [3]:
#Import Data
df = pd.read_csv('../data/Inspection.csv')

#Change Name of Column for easier searching
df.rename(columns = {'VIOLATION CODE':'CODE'}, inplace = True)

# Convert date into pandas date/time format
df['date'] = pd.to_datetime(df['INSPECTION DATE'])

df['Critical'] = np.where(df['CRITICAL FLAG'] == 'Critical', 1, 0)

In [39]:
adf = df.groupby(['CAMIS','BORO'])['date'].max()
Name_list = df.CAMIS[df.CAMIS.isna() == False].unique()
latest_inspection_dicts = {}
ldf =pd.DataFrame( df.groupby(['CAMIS','BORO'])['date'].max())

for name in Name_list:
   latest_inspection_dicts[name] = df[df.CAMIS == name].date.max()

In [77]:
df.iloc[14].date == latest_inspection_dicts[df.iloc[14].CAMIS]
df.iloc[14].Latest_Inspection

0

In [ ]:
latest_dicts = {}
for indx in range(1,len(df.CAMIS)):
   if df.iloc[indx].date == latest_inspection_dicts[df.iloc[indx].CAMIS]:
      df.iloc[indx].Latest_Inspection = 1


make a new col called latest inspection which includes the latest inspection for the resteraunt

In [50]:
df.iloc[2].date

Timestamp('1900-01-01 00:00:00')

In [43]:
df['Latest_Inspection'] = 0
df.Latest_Inspection


0         0
1         0
2         0
3         0
4         0
         ..
209574    0
209575    0
209576    0
209577    0
209578    0
Name: Latest_Inspection, Length: 209579, dtype: int64

In [64]:
#df.iloc[5].date
df[df.CAMIS == df.iloc[3].CAMIS].date.max()

Timestamp('1900-01-01 00:00:00')

In [79]:

for indx in range(len(df.CAMIS)):
   if df.iloc[indx].date == df[df.CAMIS == df.iloc[indx].CAMIS].date.max():
      df.iloc[indx].Latest_Inspection = 1

c:\Users\jakem\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


KeyboardInterrupt: 

In [80]:
df.Latest_Inspection.sum()

0

In [67]:
latest_inspection_dicts[41425803]



Timestamp('2023-01-19 00:00:00')

In [89]:
latest_inspection_dicts

{50107551: Timestamp('1900-01-01 00:00:00'),
 50131697: Timestamp('1900-01-01 00:00:00'),
 50123820: Timestamp('1900-01-01 00:00:00'),
 50119514: Timestamp('1900-01-01 00:00:00'),
 50127198: Timestamp('1900-01-01 00:00:00'),
 50131632: Timestamp('1900-01-01 00:00:00'),
 50121462: Timestamp('1900-01-01 00:00:00'),
 50112539: Timestamp('1900-01-01 00:00:00'),
 50109522: Timestamp('1900-01-01 00:00:00'),
 50129927: Timestamp('1900-01-01 00:00:00'),
 41564948: Timestamp('2018-11-09 00:00:00'),
 50113426: Timestamp('1900-01-01 00:00:00'),
 50128218: Timestamp('1900-01-01 00:00:00'),
 50104825: Timestamp('1900-01-01 00:00:00'),
 50112870: Timestamp('1900-01-01 00:00:00'),
 50120379: Timestamp('1900-01-01 00:00:00'),
 50124520: Timestamp('1900-01-01 00:00:00'),
 50124855: Timestamp('1900-01-01 00:00:00'),
 50132225: Timestamp('1900-01-01 00:00:00'),
 50116143: Timestamp('1900-01-01 00:00:00'),
 50129141: Timestamp('1900-01-01 00:00:00'),
 50121299: Timestamp('1900-01-01 00:00:00'),
 50118414:

In [ ]:
df['latestcrit'] = np.where(df['Critical'] == 1 and df.date == latest_inspection_dicts[df.CAMIS].max() , 1, 0)

In [81]:
df.CAMIS

0         50107551
1         50131697
2         50123820
3         50119514
4         50127198
            ...   
209574    50052840
209575    50072454
209576    50093635
209577    40365882
209578    50111311
Name: CAMIS, Length: 209579, dtype: int64

In [90]:
df.Latest_Inspection = df.CAMIS.map(latest_inspection_dicts)


In [115]:
ISLATEST = pd.to_datetime(df['INSPECTION DATE']) == df.Latest_Inspection
df.isLatest = ISLATEST

In [121]:
df['LatestandCrit'] = (df['isLatest'] ==True) & (df['Critical']==1 )

In [124]:
df['Critical'].sum()

113768

In [88]:
df[df.CAMIS == 40365882].Latest_Inspection

3965      0
10210     0
41436     0
77401     0
97060     0
107194    0
116266    0
146949    0
209577    0
Name: Latest_Inspection, dtype: int64

In [ ]:
if df.Latest_Inspection ==